In [1]:
import os
os.chdir('..')

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [13]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    target_column: str
    model_name: str
    alpha: float
    l1_ratio: float
    random_state: int

In [14]:
from MLOps.constants import *
from MLOps.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(list_of_directories=[self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> DataTransformationConfig:
        model_trainer_config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet
        create_directories([model_trainer_config.root_dir])
        return DataTransformationConfig(
            root_dir=model_trainer_config.root_dir,
            train_data_path=model_trainer_config.train_data_path,
            test_data_path=model_trainer_config.test_data_path,
            model_name=model_trainer_config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            random_state=params.random_state,
            target_column=schema.name
        )

In [15]:
import pandas as pd
import os
from MLOps import logger
from sklearn.linear_model import ElasticNet
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]
        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]

        model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=self.config.random_state
        )
        
        model.fit(X_train, y_train)
        
        model_file_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(model, model_file_path)

In [18]:
try:
    logger.info(">>>>>>>>> stage Model Trainer Stage started <<<<<<<<")
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    logger.info(">>>>>>>>> stage Model Trainer Stage completed <<<<<<<<")
except Exception as e: 
    logger.error(f"Error in Model Trainer Stage: {e}")
    raise e

2025-07-02 03:30:16,088 - MLOpsLogger - INFO - >>>>>>>>> stage Model Trainer Stage started <<<<<<<<
Directory already exists: artifacts
Directory already exists: artifacts\model_trainer
2025-07-02 03:30:16,207 - MLOpsLogger - INFO - >>>>>>>>> stage Model Trainer Stage completed <<<<<<<<
